**Neo4j**

In [77]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test"))

def run(str):
    with driver.session() as session:
        results = session.run(str)
        pprint.pprint(results.value())

In [70]:
# init Customer, Item, Relationships
str = """
    CREATE (ivan:Customer {name: 'Ivan'})
    CREATE (victor:Customer {name: 'Victor'})
    CREATE (stepa:Customer {name: 'Stepa'})
    
    CREATE (computer:Item {type: 'Computer', price: 1000})
    CREATE (tv:Item {type: 'TV', price: 500})
    CREATE (phone:Item {type: 'Phone', price: 600})
    CREATE (watch:Item {type: 'Watch', price: 300})
    CREATE (speaker:Item {type: 'Speaker', price: 100})
        
    CREATE (ivan)-[:VIEWED]->(computer)
    CREATE (ivan)-[:PURCHASED]->(ivan_order:Order)-[:ITEM]->(computer)
    CREATE (ivan)-[:VIEWED]->(tv)
    CREATE (ivan)-[:VIEWED]->(speaker)
    CREATE (ivan)-[:PURCHASED]->(ivan_order2:Order)-[:ITEM]->(speaker)

    CREATE (victor)-[:VIEWED]->(tv)
    CREATE (victor)-[:PURCHASED]->(victor_order:Order)-[:ITEM]->(tv)
    CREATE (victor)-[:VIEWED]->(phone)
    CREATE (victor_order)-[:ITEM]->(phone)
    
    CREATE (stepa)-[:VIEWED]->(phone)
    CREATE (stepa)-[:PURCHASED]->(stepa_order:Order {})-[:ITEM]->(phone)
    CREATE (stepa)-[:VIEWED]->(watch)
    CREATE (stepa_order)-[:ITEM]->(watch)
    CREATE (stepa)-[:VIEWED]->(speaker)
    CREATE (stepa_order)-[:ITEM]->(speaker)
    """

with driver.session() as session:
        results = session.run(str)

In [72]:
# Знайти Items які входять в конкретний Order
run("""
    MATCH (order:Order)-->(item:Item)
    WHERE id(order) = 3
    RETURN item
    """)

[<Node id=10 labels=frozenset({'Item'}) properties={'type': 'Phone', 'price': 600}>,
 <Node id=9 labels=frozenset({'Item'}) properties={'type': 'TV', 'price': 500}>]


In [73]:
# Підрахувати вартість конкретного Order
run("""
    MATCH (order:Order)-->(item:Item)
    WHERE id(order) = 3
    RETURN sum(item.price)
    """)

[1100]


In [75]:
# Знайти всі Orders конкретного Customer
run("""
    MATCH (customer:Customer)-->(order:Order)
    WHERE customer.name = 'Ivan'
    RETURN order
    """)

[<Node id=2 labels=frozenset({'Order'}) properties={}>,
 <Node id=1 labels=frozenset({'Order'}) properties={}>]


In [78]:
# Знайти всі Items куплені конкретним Customer (через Order)
run("""
    MATCH (customer:Customer)-->(order:Order)-->(item:Item)
    WHERE customer.name = 'Ivan'
    RETURN item
    """)

[<Node id=0 labels=frozenset({'Item'}) properties={'type': 'Speaker', 'price': 100}>,
 <Node id=8 labels=frozenset({'Item'}) properties={'type': 'Computer', 'price': 1000}>]


In [79]:
# Знайти кількість Items куплені конкретним Customer (через Order)
run("""
    MATCH (customer:Customer)-->(order:Order)-->(item:Item)
    WHERE customer.name = 'Ivan'
    RETURN count(item)
    """)

[2]


In [82]:
# Знайти для Customer на яку суму він придбав товарів (через Order)
run("""
    MATCH (customer:Customer)-->(order:Order)-->(item:Item)
    WHERE customer.name = 'Ivan'
    RETURN sum(item.price)
    """)

[1100]


In [156]:
# Знайті скільки разів кожен товар був придбаний, відсортувати за цим значенням
run("""
    MATCH (customer:Customer)-[p:PURCHASED]->(order:Order)-->(item:Item)
    RETURN distinct(item), count(p) as x
    ORDER BY x DESC;
    """)

#  todo todo todo

[<Node id=0 labels=frozenset({'Item'}) properties={'type': 'Speaker', 'price': 100}>,
 <Node id=10 labels=frozenset({'Item'}) properties={'type': 'Phone', 'price': 600}>,
 <Node id=8 labels=frozenset({'Item'}) properties={'type': 'Computer', 'price': 1000}>,
 <Node id=9 labels=frozenset({'Item'}) properties={'type': 'TV', 'price': 500}>,
 <Node id=11 labels=frozenset({'Item'}) properties={'type': 'Watch', 'price': 300}>]


In [103]:
# Знайти всі Items переглянуті (view) конкретним Customer
run("""
    MATCH (customer:Customer)-->(item:Item)
    WHERE customer.name = 'Ivan'
    RETURN item
    """)

[<Node id=0 labels=frozenset({'Item'}) properties={'type': 'Speaker', 'price': 100}>,
 <Node id=9 labels=frozenset({'Item'}) properties={'type': 'TV', 'price': 500}>,
 <Node id=8 labels=frozenset({'Item'}) properties={'type': 'Computer', 'price': 1000}>]


In [113]:
# Знайти інші Items що купувались разом з конкретним Item 
# (тобто всі Items що входять до Order-s разом з даними Item)
run("""
    MATCH (:Item {type: 'Phone'})<--(order:Order)
    MATCH (ord:Order)-->(item:Item)
    WHERE id(ord) = id(order)
    RETURN DISTINCT item
    """)

[<Node id=0 labels=frozenset({'Item'}) properties={'type': 'Speaker', 'price': 100}>,
 <Node id=11 labels=frozenset({'Item'}) properties={'type': 'Watch', 'price': 300}>,
 <Node id=10 labels=frozenset({'Item'}) properties={'type': 'Phone', 'price': 600}>,
 <Node id=9 labels=frozenset({'Item'}) properties={'type': 'TV', 'price': 500}>]


In [114]:
# Знайти Customers які купили даний конкретний Item
run("""
    MATCH (:Item {type: 'Phone'})<--(:Order)<--(cust:Customer)
    RETURN cust
    """)

[<Node id=7 labels=frozenset({'Customer'}) properties={'name': 'Stepa'}>,
 <Node id=6 labels=frozenset({'Customer'}) properties={'name': 'Victor'}>]


In [144]:
# Знайти для певного Customer(а) товари, 
# які він переглядав, але не купив
run("""
    MATCH (customer:Customer {name: 'Ivan'})-[:VIEWED]->(item:Item)
    WHERE NOT EXISTS {
        MATCH  (customer)-->(order:Order)-->(item)
    }
    RETURN item
    """)

[<Node id=9 labels=frozenset({'Item'}) properties={'type': 'TV', 'price': 500}>]


In [80]:
driver.close()